
###([Mastering the Huggingface CLIP Model: How to Extract Embeddings and Calculate Similarity for Text and Images](https://codeandlife.com/2023/01/26/mastering-the-huggingface-clip-model-how-to-extract-embeddings-and-calculate-similarity-for-text-and-images/))

In [22]:
from datasets import load_from_disk
import torch
from transformers import AutoTokenizer, CLIPModel

In [3]:
HF_DATASET_DIR = "/workspaces/ASE-Model-Retrieval/data/imagenet/.cache/hf_datasets"

In [ ]:
hf_dataset_dict = load_from_disk(HF_DATASET_DIR)
hf_dataset_dict

In [21]:
tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-large-patch14")
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)

In [ ]:
def get_text_features(labels):
    text_inputs = tokenizer(
        [f"a photo of a {label}" for label in labels], padding=True, return_tensors="pt"
    ).to(device)
    with torch.no_grad():
        text_features = model.get_text_features(**text_inputs)
    return text_features

def get_image_features(images):
    

In [27]:
def process_dataset(dataset_name, dataset):
    print(f"Processing dataset: {dataset_name}")

    unique_labels = list(set(dataset["label"]))
    text_features = get_text_features(unique_labels)
    print(text_features.shape)

for dataset_name, dataset in hf_dataset_dict.items():
    process_dataset(dataset_name, dataset)

Processing dataset: subclass_EntleBucher-German_shepherd-bluetick-croquet_ball-tench
torch.Size([5, 768])
Processing dataset: subclass_Border_terrier-Lakeland_terrier-Lhasa-Norfolk_terrier-Sussex_spaniel
torch.Size([5, 768])
Processing dataset: subclass_Border_collie-Pomeranian-cairn-kelpie-terrapin
torch.Size([5, 768])
Processing dataset: subclass_English_foxhound-Scottie-baseball-giant_schnauzer-minibus
torch.Size([5, 768])
Processing dataset: subclass_Norwich_terrier-banana-bloodhound-kite-minivan
torch.Size([5, 768])
Processing dataset: subclass_Norwich_terrier-Scottie-dingo-minivan-red_wolf
torch.Size([5, 768])
Processing dataset: subclass_Bouvier-Hungarian_pointer-jackfruit-leopard-miniature_poodle
torch.Size([5, 768])
Processing dataset: subclass_custard_apple-flat_coated_retriever-minibus-red_wolf-sports_car
torch.Size([5, 768])
Processing dataset: subclass_Italian_greyhound-Labrador_retriever-Welsh_springer_spaniel-bloodhound-snow_leopard
torch.Size([5, 768])
Processing datase

KeyboardInterrupt: 